In [1]:
import torch
import torchani
import glob
import cclib
import pandas as pd

In [2]:
# path to ANI-2x subdirectories
path = '/home/dakota/Projects/ml-benchmark/ani-2x'

In [3]:
import numpy as np
from math import sqrt
from chemreps.utils.molecule import Molecule
import re

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


In [4]:
# for ANI-2x from sae
atom_selfE = {
'H': -0.5975740895940036,
'C': -38.08963824205658,
'N': -54.71039678680539,
'O': -75.19115551439664,
'S': -398.1787966261814,
'F': -99.79839227891235,
'Cl': -460.206968972609
}

atom_conv = {1: 0, 6: 1, 7: 2, 8: 3, 16: 4, 9: 5, 17: 6}
# atom_conv = {0: 1, 1: 6, 2: 7, 3: 6, 4: 16, 5: 9, 6: 17}

device = torch.device('cpu')
const_file = path + '/rHCNOSFCl-5.1R_16-3.5A_a8-4.params'  # noqa: E501
consts = torchani.neurochem.Constants(const_file)
aev_computer = torchani.AEVComputer(**consts)
sae_file = path + '/sae_linfit.dat'  # noqa: E501
energy_shifter = torchani.neurochem.load_sae(sae_file)
model_prefix = path + '/train'  # noqa: E501
ensemble = torchani.neurochem.load_model_ensemble(consts.species, model_prefix, 8)  # noqa: E501

model = torch.nn.Sequential(aev_computer, ensemble, energy_shifter).to(device)


results = []
for out in sorted(glob.iglob('molecules/*/sdf/*.sdf'), key=numericalSort):
    try:
        name = out.split('es/')[1].split('/sdf')[0]
        pt = out.split('sdf/')[1].split('.')[0]
        mol = Molecule(out)
        atoms = []
        for i in mol.at_num:
            atoms.append(atom_conv[i])
        coords = mol.xyz.tolist()

        coordinates = torch.tensor([coords],
                               requires_grad=True, device=device)
        species = torch.tensor([atoms], device=device)

        _, energy = model((species, coordinates))
        
        # convert hartree to eV
        e_ev = energy.item() * 27.2114

        d = {}
        d.update({'name': name})
        d.update({'point': pt})
        d.update({'ani2_energy': e_ev})
    #         d.update({'force': list(force.cpu().data.numpy())})
        results.append(d)

        print('Name:{}/{} \tEnergy:{} \tAtomE:{}'.format(name, pt, energy.item(), -627.509469*(energy.item())))
    except:
        print('File {} did not run'.format(out))

Name:C6H6-C/1 	Energy:-230.06175974758716 	AtomE:144365.93269641398
Name:C6H6-C/2 	Energy:-231.1171939745769 	AtomE:145028.22766775676
Name:C6H6-C/3 	Energy:-231.81451673062892 	AtomE:145465.80430012857
Name:C6H6-C/4 	Energy:-232.09666966090353 	AtomE:145642.85793558197
Name:C6H6-C/5 	Energy:-232.17877986656876 	AtomE:145694.38286713845
Name:C6H6-C/6 	Energy:-232.13602330444755 	AtomE:145667.5527195455
Name:C6H6-C/7 	Energy:-232.01329082252832 	AtomE:145590.5369249873
Name:C6H6-C/8 	Energy:-231.8598408773765 	AtomE:145494.24563138702
Name:C6H6-C/9 	Energy:-231.65787131341548 	AtomE:145367.50781755167
Name:C6H6-C/10 	Energy:-231.39826289566608 	AtomE:145204.6010771818
Name:C6H6-C/11 	Energy:-231.20523879679294 	AtomE:145083.47662739374
Name:C6H6-C/12 	Energy:-231.13100603970142 	AtomE:145036.89486940883
Name:C6H6-C/13 	Energy:-231.10178748520465 	AtomE:145018.5599497916
Name:C6H6-C/14 	Energy:-231.04778698834033 	AtomE:144984.67412667855
Name:C6H6-C/15 	Energy:-230.95945266636463 	AtomE

Name:CH3OH/38 	Energy:-115.38410026231955 	AtomE:72404.6154866509
Name:CH3OH/39 	Energy:-115.38441473642538 	AtomE:72404.81282213006
Name:CH3OH/40 	Energy:-115.38457528153609 	AtomE:72404.91356570723
Name:CH3OH/41 	Energy:-115.38463542262267 	AtomE:72404.95130480854
Name:CH3OH/42 	Energy:-115.38464430371474 	AtomE:72404.9568777779
Name:CH3OH/43 	Energy:-115.38464430371474 	AtomE:72404.9568777779
Name:CH3OH/44 	Energy:-115.38464433351706 	AtomE:72404.95689647914
Name:CH3OH/45 	Energy:-115.38464433351706 	AtomE:72404.95689647914
Name:CH4/1 	Energy:-39.35565386399498 	AtomE:24696.045458343287
Name:CH4/2 	Energy:-39.9968131004261 	AtomE:25098.378950340626
Name:CH4/3 	Energy:-40.32824836358299 	AtomE:25306.35771633208
Name:CH4/4 	Energy:-40.460771011783386 	AtomE:25389.516932934785
Name:CH4/5 	Energy:-40.49913710415235 	AtomE:25413.592019184834
Name:CH4/6 	Energy:-40.481201915752486 	AtomE:25402.337518635624
Name:CH4/7 	Energy:-40.41333596095314 	AtomE:25359.75098937631
Name:CH4/8 	Energy:-

Name:H2CCH2/14 	Energy:-78.2013526389044 	AtomE:49072.08926952064
Name:H2CCH2/15 	Energy:-78.14299057215864 	AtomE:49035.46652000727
Name:H2CCH2/16 	Energy:-78.10376547068769 	AtomE:49010.85239741176
Name:H2CCH2/17 	Energy:-78.0812447795313 	AtomE:48996.72045046271
Name:H2CCH2/18 	Energy:-78.06871579379255 	AtomE:48988.85839327468
Name:H2CCH2/19 	Energy:-78.0654101201933 	AtomE:48986.78405178973
Name:H2CCH2/20 	Energy:-78.06955854386503 	AtomE:48989.38722692515
Name:H2CCH2/21 	Energy:-78.08079205245191 	AtomE:48996.43635993352
Name:H2CCH2/22 	Energy:-78.09679101199323 	AtomE:49006.475858539845
Name:H2CCH2/23 	Energy:-78.11414353341276 	AtomE:49017.36473004162
Name:H2CCH2/24 	Energy:-78.13003887027914 	AtomE:49027.33920443822
Name:H2CCH2/25 	Energy:-78.14241520852262 	AtomE:49035.10547387755
Name:H2CCH2/26 	Energy:-78.14941854566747 	AtomE:49039.50013425054
Name:H2CCH2/27 	Energy:-78.15104235500509 	AtomE:49040.51908998575
Name:H2CCH2/28 	Energy:-78.1510287949484 	AtomE:49040.5105809217

Name:HCN/21 	Energy:-93.10916527323012 	AtomE:58426.882859637866
Name:HCN/22 	Energy:-93.12011858038191 	AtomE:58433.75616359249
Name:HCN/23 	Energy:-93.12864773682837 	AtomE:58439.10829002522
Name:HCN/24 	Energy:-93.13489463142638 	AtomE:58443.02827553731
Name:HCN/25 	Energy:-93.1452601175714 	AtomE:58449.5327162441
Name:HCN/26 	Energy:-93.15354483536963 	AtomE:58454.73145511049
Name:HCN/27 	Energy:-93.1526345234323 	AtomE:58454.16022575006
Name:HCN/28 	Energy:-93.14659266881232 	AtomE:58450.36890476571
Name:HCN/29 	Energy:-93.13779651335959 	AtomE:58444.849233928326
Name:HCN/30 	Energy:-93.12705292515044 	AtomE:58438.10753059605
Name:HCN/31 	Energy:-93.1148901801515 	AtomE:58430.47529294018
Name:HCN/32 	Energy:-93.1024110536981 	AtomE:58422.644522925824
Name:HCN/33 	Energy:-93.09126599840407 	AtomE:58415.65089519629
Name:HCN/34 	Energy:-93.08413203768019 	AtomE:58411.17426729058
Name:HCN/35 	Energy:-93.08151563219313 	AtomE:58409.53244807271
Name:HCN/36 	Energy:-93.08330678157095 	At

Name:NH3/17 	Energy:-56.008081179322836 	AtomE:35145.601280545765
Name:NH3/18 	Energy:-55.988077353896735 	AtomE:35133.04869067467
Name:NH3/19 	Energy:-55.96645177549326 	AtomE:35119.47843545388
Name:NH3/20 	Energy:-55.940028321208594 	AtomE:35102.89746768656
Name:NH3/21 	Energy:-55.91353942102396 	AtomE:35086.27543199731
Name:NH3/22 	Energy:-55.89983422702753 	AtomE:35077.67529299007
Name:NH3/23 	Energy:-55.8971393222233 	AtomE:35075.984214707365
Name:NH3/24 	Energy:-55.90762175268137 	AtomE:35082.56203907794
Name:NH3/25 	Energy:-55.93911160177195 	AtomE:35102.32221755965
Name:NH3/26 	Energy:-55.97136003917658 	AtomE:35122.55841739151
Name:NH3/27 	Energy:-55.990477454127905 	AtomE:35134.554776296274
Name:NH3/28 	Energy:-56.0037945622822 	AtomE:35142.91138776279
Name:NH3/29 	Energy:-56.0135251695534 	AtomE:35149.01743596458
Name:NH3/30 	Energy:-56.02315438932383 	AtomE:35155.05986254961
Name:NH3/31 	Energy:-56.03742716855013 	AtomE:35164.01616666307
Name:NH3/32 	Energy:-56.058783780994

In [5]:
df = pd.DataFrame(results, columns=['name', 'point', 'ani2_energy'])
df

,name,point,ani2_energy
0,C6H6-C,1,-6260.302569
1,C6H6-C,2,-6289.022412
2,C6H6-C,3,-6307.997541
3,C6H6-C,4,-6315.675317
4,C6H6-C,5,-6317.909650
...,...,...,...
580,NH3,41,-1529.949810
581,NH3,42,-1530.330988
582,NH3,43,-1530.534665
583,NH3,44,-1530.596923


In [6]:
df.to_csv('data/ani2-results.csv', index=False)